## Setup Environment and Libraries

### Subtask:
Install necessary libraries such as MediaPipe, OpenCV, NumPy, etc., and configure the development environment.


In [ ]:
import sys
import subprocess

def install_package(package):
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
        print(f"Successfully installed {package}")
    except subprocess.CalledProcessError as e:
        print(f"Failed to install {package}: {e}")

install_package("mediapipe")
install_package("opencv-python")
install_package("numpy")
install_package("pandas")
install_package("matplotlib")
install_package("seaborn")
install_package("scipy")

print("All specified libraries have been installed.")

## Video Input Handling

### Subtask:
Develop functionality to allow users to upload their 5-10 second workout video clips and load them for processing.


## Video Input Handling

### Subtask:
Develop functionality to allow users to upload their 5-10 second workout video clips and load them for processing.

### Instructions to the user:

1.  Run the next code cell to upload your video file. A file selection widget will appear.
2.  Select your workout video (preferably 5-10 seconds long) and upload it.
3.  After successful upload, the system will process the video and display its basic properties.

In [ ]:
from google.colab import files
import cv2
import os

# 1. Use files.upload() to create an upload widget
uploaded = files.upload()

video_path = None
if uploaded:
    # Assuming only one file is uploaded for simplicity
    video_filename = list(uploaded.keys())[0]
    video_path = f"/content/{video_filename}"
    print(f"Uploaded file: {video_filename}")

    # 3. Load the uploaded video file using cv2.VideoCapture()
    cap = cv2.VideoCapture(video_path)

    # 4. Verify that the video has been successfully opened
    if not cap.isOpened():
        print("Error: Could not open video.")
    else:
        # 5. Print basic video properties
        frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        fps = cap.get(cv2.CAP_PROP_FPS)
        frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

        print(f"\nVideo Properties:")
        print(f"  Resolution: {frame_width}x{frame_height}")
        print(f"  FPS: {fps}")
        print(f"  Total Frames: {frame_count}")
        print(f"  Duration: {frame_count / fps:.2f} seconds")

        # Release the video capture object
        cap.release()
else:
    print("No file uploaded.")

## Pose Keypoint Detection

### Subtask:
Implement a pose estimation model (e.g., MediaPipe Pose) to detect and extract 3D keypoints from each frame of the uploaded video.


In [ ]:
import mediapipe as mp
import cv2
import numpy as np

# 1. Import mediapipe.solutions.pose
mp_pose = mp.solutions.pose

# 2. Initialize the MediaPipe Pose model
# Set static_image_mode to False for video processing to track objects across frames
# Set model_complexity to 1 or 2 for higher accuracy if needed, 0 is fastest.
pose = mp_pose.Pose(static_image_mode=False, model_complexity=1, min_detection_confidence=0.5, min_tracking_confidence=0.5)

# 3. Create an empty list to store detected keypoints
keypoints_data = []

# Ensure video_path is available from previous step
if 'video_path' not in locals() or video_path is None:
    print("Error: video_path not found. Please upload a video first.")
else:
    # 4. Re-open the uploaded video
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print("Error: Could not re-open video for keypoint detection.")
    else:
        frame_count_processed = 0
        print("Starting keypoint detection...")
        while cap.isOpened():
            # 5. Loop through each frame of the video
            ret, frame = cap.read()
            if not ret:
                break

            # 6. Convert the frame from BGR to RGB
            image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            # 7. Process the RGB frame with MediaPipe Pose model
            results = pose.process(image_rgb)

            # 8. If results.pose_landmarks are detected, append them to keypoints_data
            if results.pose_landmarks:
                # Store all landmarks for the current frame
                frame_landmarks = []
                for landmark in results.pose_landmarks.landmark:
                    frame_landmarks.append({
                        'x': landmark.x,
                        'y': landmark.y,
                        'z': landmark.z,
                        'visibility': landmark.visibility
                    })
                keypoints_data.append(frame_landmarks)
                frame_count_processed += 1

        # 9. Release the video capture object
        cap.release()
        pose.close()

        # 10. Print the total number of frames for which keypoints were detected
        print(f"Keypoint detection complete. Detected keypoints for {len(keypoints_data)} frames.")


In [ ]:
import pandas as pd

# Convert the list of keypoints for each frame into a DataFrame
# Each row represents a frame, and columns represent keypoint attributes (x, y, z, visibility) for each landmark

if keypoints_data:
    # Flatten the list of lists into a list of dictionaries where each dict represents a landmark
    # and add a frame_id to link back to the original frame.
    flattened_data = []
    for frame_idx, frame_landmarks in enumerate(keypoints_data):
        for landmark_idx, landmark in enumerate(frame_landmarks):
            flattened_data.append({
                'frame_id': frame_idx,
                'landmark_id': landmark_idx,
                'x': landmark['x'],
                'y': landmark['y'],
                'z': landmark['z'],
                'visibility': landmark['visibility']
            })

    keypoints_df = pd.DataFrame(flattened_data)

    print(f"Successfully converted {len(keypoints_data)} frames of keypoints into a DataFrame.")
    print(f"DataFrame shape: {keypoints_df.shape}")
    print("\nSample of keypoint data:")
    display(keypoints_df.head())
else:
    print("No keypoints were detected or keypoints_data is empty.")

## Keypoint Data Extraction and Smoothing

### Subtask:
Extract time-series data for relevant keypoints (e.g., elbows, wrists, shoulders, hips) and apply smoothing techniques if necessary to reduce jitter.


In [ ]:
import pandas as pd

# 1. Define a mapping from landmark_id to human-readable names for relevant keypoints
# Based on MediaPipe Pose documentation
keypoint_map = {
    11: 'left_shoulder',
    12: 'right_shoulder',
    13: 'left_elbow',
    14: 'right_elbow',
    15: 'left_wrist',
    16: 'right_wrist',
    23: 'left_hip',
    24: 'right_hip'
}

# 2. Filter the keypoints_df DataFrame to include only the rows corresponding to these relevant keypoints
relevant_keypoints_df = keypoints_df[keypoints_df['landmark_id'].isin(keypoint_map.keys())].copy()

# Add a 'keypoint_name' column to the filtered DataFrame
relevant_keypoints_df['keypoint_name'] = relevant_keypoints_df['landmark_id'].map(keypoint_map)

# Prepare for pivoting: create a 'variable' column for x, y, z coordinates
# This will allow pivoting multiple columns simultaneously

# Temporarily melt the x,y,z columns into rows for easier pivoting
melted_keypoints = relevant_keypoints_df.melt(id_vars=['frame_id', 'keypoint_name'],
                                                value_vars=['x', 'y', 'z'],
                                                var_name='coordinate_type',
                                                value_name='value')

# Combine keypoint_name and coordinate_type to create new column names
melted_keypoints['new_column_name'] = melted_keypoints['keypoint_name'] + '_' + melted_keypoints['coordinate_type']

# 3. Pivot the filtered and melted DataFrame
# Each row represents a frame, and columns represent the x, y, and z coordinates for each selected keypoint
pivoted_keypoints_df = melted_keypoints.pivot_table(index='frame_id', columns='new_column_name', values='value')

# Reset index to make 'frame_id' a regular column if needed, but often better as index for time series
# pivoted_keypoints_df = pivoted_keypoints_df.reset_index()

print(f"Original keypoints_df shape: {keypoints_df.shape}")
print(f"Filtered relevant_keypoints_df shape: {relevant_keypoints_df.shape}")
print(f"Pivoted keypoints DataFrame shape: {pivoted_keypoints_df.shape}")
print("\nSample of pivoted keypoints data:")
display(pivoted_keypoints_df.head())

In [ ]:
import pandas as pd

# 4. Apply smoothing techniques
# A common approach is a rolling mean or a Savitzky-Golay filter.
# For rolling mean, a window size of 5-10 frames is often a good starting point for ~30 FPS videos.
# For Savitzky-Golay, a window length and polynomial order are needed.

# Let's use a rolling mean for smoothing.
# We will use a window size that corresponds to approximately 0.2-0.3 seconds for a 30 FPS video
# window_size = int(fps * 0.25) # 0.25 seconds smoothing
# Using a fixed window size for now, can be made dynamic based on FPS
window_size = 5 # Example: 5 frames

# Apply rolling mean to all coordinate columns
smoothed_keypoints_df = pivoted_keypoints_df.rolling(window=window_size, center=True, min_periods=1).mean()

# Alternatively, using Savitzky-Golay filter for potentially better preservation of signal features
# Note: Savitzky-Golay requires non-NaN data, so we might need to handle NaNs introduced by rolling mean if applied after.
# For this example, applying directly to the pivoted_keypoints_df columns.
# window_length must be odd and greater than polyorder

# from scipy.signal import savgol_filter # Removed this line due to ImportError
# savgol_window_length = 5 # Must be odd
# savgol_poly_order = 2

# smoothed_keypoints_df_savgol = pivoted_keypoints_df.copy()
# for col in pivoted_keypoints_df.columns:
#     # Fill NaN values to apply savgol_filter, common strategy is forward fill or interpolate
#     # For simplicity, if a column has many NaNs, it might indicate bad detection for that keypoint/frame
#     # Here, we assume no NaNs in the middle, only at start/end due to min_periods if applied previously
#     smoothed_keypoints_df_savgol[col] = savgol_filter(pivoted_keypoints_df[col].fillna(method='ffill'), savgol_window_length, savgol_poly_order)

print(f"Smoothed keypoints DataFrame shape: {smoothed_keypoints_df.shape}")
print(f"Smoothing applied with a rolling mean window of {window_size} frames.")
print("\nSample of smoothed keypoints data:")
display(smoothed_keypoints_df.head())

## Define Exercise-Specific Form Rules

### Subtask:
Develop rule-based logic for various exercises (e.g., bicep curl, lateral raise) including rules for angles (e.g., elbow angle), alignments (e.g., wrist-shoulder), and symmetry/posture (e.g., back angle).


In [ ]:
import numpy as np
import pandas as pd

def calculate_angle(df, keypoint1_name, keypoint2_name, keypoint3_name):
    """
    Calculates the 3D angle (in degrees) between three keypoints for each frame in the DataFrame.

    Args:
        df (pd.DataFrame): DataFrame containing smoothed keypoint coordinates.
        keypoint1_name (str): Name of the first keypoint (e.g., 'right_shoulder').
        keypoint2_name (str): Name of the second (middle) keypoint (e.g., 'right_elbow').
        keypoint3_name (str): Name of the third keypoint (e.g., 'right_wrist').

    Returns:
        pd.Series: A Series containing the angles in degrees for each frame.
    """
    angles = []
    for index, row in df.iterrows():
        try:
            # Extract coordinates for the three keypoints
            p1 = np.array([row[f'{keypoint1_name}_x'], row[f'{keypoint1_name}_y'], row[f'{keypoint1_name}_z']])
            p2 = np.array([row[f'{keypoint2_name}_x'], row[f'{keypoint2_name}_y'], row[f'{keypoint2_name}_z']])
            p3 = np.array([row[f'{keypoint3_name}_x'], row[f'{keypoint3_name}_y'], row[f'{keypoint3_name}_z']])

            # Calculate vectors
            v1 = p1 - p2
            v2 = p3 - p2

            # Calculate the cosine of the angle using the dot product formula
            # Handle division by zero if a vector has zero magnitude
            dot_product = np.dot(v1, v2)
            magnitude_v1 = np.linalg.norm(v1)
            magnitude_v2 = np.linalg.norm(v2)

            if magnitude_v1 == 0 or magnitude_v2 == 0:
                angle = np.nan # Undefined angle if a vector is zero
            else:
                cosine_angle = dot_product / (magnitude_v1 * magnitude_v2)
                # Clamp the value to the valid range [-1, 1] to avoid issues with floating point inaccuracies
                cosine_angle = np.clip(cosine_angle, -1.0, 1.0)
                angle = np.degrees(np.arccos(cosine_angle))
            angles.append(angle)
        except KeyError as e:
            print(f"Missing keypoint coordinate: {e}. Skipping frame {index}.")
            angles.append(np.nan)
        except Exception as e:
            print(f"An error occurred during angle calculation for frame {index}: {e}. Skipping frame.")
            angles.append(np.nan)
    return pd.Series(angles, index=df.index)

print("Defined calculate_angle function.")


In [ ]:
import pandas as pd
import numpy as np

# Create a new DataFrame to store computed angles
angles_df = pd.DataFrame(index=smoothed_keypoints_df.index)

# 1. Calculate Right Elbow Angle: (Right Shoulder - Right Elbow - Right Wrist)
angles_df['right_elbow_angle'] = calculate_angle(
    smoothed_keypoints_df, 'right_shoulder', 'right_elbow', 'right_wrist'
)

# 2. Calculate Left Elbow Angle: (Left Shoulder - Left Elbow - Left Wrist)
angles_df['left_elbow_angle'] = calculate_angle(
    smoothed_keypoints_df, 'left_shoulder', 'left_elbow', 'left_wrist'
)

# 3. Calculate Right Shoulder Angle: (Right Hip - Right Shoulder - Right Elbow)
angles_df['right_shoulder_angle'] = calculate_angle(
    smoothed_keypoints_df, 'right_hip', 'right_shoulder', 'right_elbow'
)

# 4. Calculate Left Shoulder Angle: (Left Hip - Left Shoulder - Left Elbow)
angles_df['left_shoulder_angle'] = calculate_angle(
    smoothed_keypoints_df, 'left_hip', 'left_shoulder', 'left_elbow'
)

# 5. Calculate Right Knee Angle: (Right Hip - Right Knee - Right Ankle)
# For bicep curls, knee angle might not be critical, but included for completeness in other exercises.
# Assuming keypoints for knee and ankle exist, if not, this will result in NaNs
# Keypoint IDs for knee and ankle: Right Hip (24), Right Knee (26), Right Ankle (28)
# Left Hip (23), Left Knee (25), Left Ankle (27)
# This assumes these keypoints were included in keypoint_map for pivoted_keypoints_df,
# but only specific ones were, so these will likely produce KeyError unless we update keypoint_map.
# Let's add them to the map for this step for future use if needed, but for now we focus on upper body for bicep curl
# To avoid error and stay focused on the bicep curl for now, I will comment out knee/ankle

# Display the first few rows of the angles DataFrame
print("Calculated angles for relevant keypoints:")
display(angles_df.head())

print(f"Angles DataFrame shape: {angles_df.shape}")

In [ ]:
bicep_curl_rules = {
    'start_position': {
        'right_elbow_angle': {'min': 160, 'max': 180, 'deviation_message': 'Right arm not fully extended at start.'},
        'left_elbow_angle': {'min': 160, 'max': 180, 'deviation_message': 'Left arm not fully extended at start.'},
        'right_shoulder_angle': {'min': 15, 'max': 45, 'deviation_message': 'Right shoulder not stable, arm swinging forward/backward.'},
        'left_shoulder_angle': {'min': 15, 'max': 45, 'deviation_message': 'Left shoulder not stable, arm swinging forward/backward.'}
    },
    'peak_contraction': {
        'right_elbow_angle': {'min': 30, 'max': 60, 'deviation_message': 'Right elbow not fully contracted.'},
        'left_elbow_angle': {'min': 30, 'max': 60, 'deviation_message': 'Left elbow not fully contracted.'}
    },
    'full_extension': {
        'right_elbow_angle': {'min': 160, 'max': 180, 'deviation_message': 'Right arm not fully extended at bottom.'},
        'left_elbow_angle': {'min': 160, 'max': 180, 'deviation_message': 'Left arm not fully extended at bottom.'}
    },
    'general_form': {
        'right_shoulder_angle': {'min': 10, 'max': 50, 'deviation_message': 'Right shoulder moving too much (swinging).'},
        'left_shoulder_angle': {'min': 10, 'max': 50, 'deviation_message': 'Left shoulder moving too much (swinging).'}
    }
}

print("Defined bicep curl form rules.")

In [ ]:
import pandas as pd
import numpy as np

# 1. Initialize an empty list to store frame-wise feedback
frame_feedback = []

def evaluate_frame_bicep_curl(frame_angles, bicep_curl_rules):
    """
    Evaluates the form correctness for a single frame of a bicep curl exercise.
    Identifies the exercise phase and checks angles against general and phase-specific rules.

    Args:
        frame_angles (pd.Series): A Series containing angle values for a single frame.
        bicep_curl_rules (dict): A dictionary defining the rules for bicep curl form.

    Returns:
        list: A list of deviation messages for the current frame.
    """
    deviations = []

    right_elbow_angle = frame_angles.get('right_elbow_angle')
    left_elbow_angle = frame_angles.get('left_elbow_angle')

    # Determine the current exercise phase based on elbow angles
    phase_rules_to_check = {}
    identified_phase = None

    if pd.notna(right_elbow_angle) and pd.notna(left_elbow_angle):
        # Check for Peak Contraction phase
        peak_right_min = bicep_curl_rules['peak_contraction']['right_elbow_angle']['min']
        peak_right_max = bicep_curl_rules['peak_contraction']['right_elbow_angle']['max']
        peak_left_min = bicep_curl_rules['peak_contraction']['left_elbow_angle']['min']
        peak_left_max = bicep_curl_rules['peak_contraction']['left_elbow_angle']['max']

        if (peak_right_min <= right_elbow_angle <= peak_right_max) and \
           (peak_left_min <= left_elbow_angle <= peak_left_max):
            phase_rules_to_check = bicep_curl_rules['peak_contraction']
            identified_phase = 'peak_contraction'
        else:
            # If not peak contraction, check for extended positions (start or full extension)
            # Since rules for start_position and full_extension are identical for elbow angles, we can check against one.
            extended_right_min = bicep_curl_rules['start_position']['right_elbow_angle']['min']
            extended_right_max = bicep_curl_rules['start_position']['right_elbow_angle']['max']
            extended_left_min = bicep_curl_rules['start_position']['left_elbow_angle']['min']
            extended_left_max = bicep_curl_rules['start_position']['left_elbow_angle']['max']

            if (extended_right_min <= right_elbow_angle <= extended_right_max) and \
               (extended_left_min <= left_elbow_angle <= extended_left_max):
                phase_rules_to_check = bicep_curl_rules['start_position'] # Apply start/full extension rules
                identified_phase = 'extended_position' # Could be start or full extension

    # Check 'general_form' rules
    for angle_name, rule_limits in bicep_curl_rules['general_form'].items():
        angle_value = frame_angles.get(angle_name)
        if pd.notna(angle_value):
            if not (rule_limits['min'] <= angle_value <= rule_limits['max']):
                deviations.append(f"{angle_name.replace('_', ' ').title()}: {rule_limits['deviation_message']}")

    # Check phase-specific rules if a phase was identified
    for angle_name, rule_limits in phase_rules_to_check.items():
        angle_value = frame_angles.get(angle_name)
        if pd.notna(angle_value):
            if not (rule_limits['min'] <= angle_value <= rule_limits['max']):
                # Avoid duplicate messages if general form already caught it
                deviation_msg = f"{angle_name.replace('_', ' ').title()}: {rule_limits['deviation_message']}"
                if deviation_msg not in deviations:
                    deviations.append(deviation_msg)

    return deviations

# 3. Iterate through each row (frame) of the angles_df DataFrame
for frame_id, frame_angles_series in angles_df.iterrows():
    deviations = evaluate_frame_bicep_curl(frame_angles_series, bicep_curl_rules)
    frame_feedback.append({'frame_id': frame_id, 'deviations': deviations})

# 4. Print the first 10 entries of the frame_feedback list
print("Sample of frame-wise feedback (first 10 entries):")
for entry in frame_feedback[:10]:
    print(f"Frame {entry['frame_id']}: {entry['deviations']}")

## Generate Visual Feedback Video

### Subtask:
Create a new video with overlays on the original user video, showing detected keypoints, relevant angles/lines, and real-time textual feedback on form correctness (e.g., 'Elbow too low!', 'Good form').


In [ ]:
import cv2
import numpy as np
import mediapipe as mp
from IPython.display import Video, display

# 1. Re-open the original user-uploaded video
cap = cv2.VideoCapture(video_path)

# Ensure video properties are available (from previous steps)
if 'fps' not in locals() or 'frame_width' not in locals() or 'frame_height' not in locals():
    # Fallback to re-calculating if not found (should be present from previous upload step)
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    if not cap.isOpened():
        print("Error: Could not open video to re-calculate properties.")

# 2. Initialize MediaPipe drawing utilities (we won't use draw_landmarks directly, but we need POSE_CONNECTIONS)
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# 3. Create a cv2.VideoWriter object to save the output video
output_video_path = '/content/workout_feedback_video.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v') # Codec for .mp4 files
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

print(f"Generating feedback video... (Output: {output_video_path})")

frame_idx = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    annotated_frame = frame.copy()

    # Define a helper function to get pixel coordinates for any landmark
    def get_pixel_coords_all_landmarks(landmark_data, frame_width, frame_height):
        lm_dict = {}
        # landmark_data is a list of dictionaries for the current frame
        for idx, lm in enumerate(landmark_data):
            if lm['visibility'] > 0.5: # Only consider visible landmarks
                lm_dict[idx] = (int(lm['x'] * frame_width), int(lm['y'] * frame_height))
        return lm_dict

    # 4a. Manually draw landmarks and connections on the frame
    if frame_idx < len(keypoints_data):
        current_frame_keypoints_raw = keypoints_data[frame_idx]
        all_landmarks_pixel = get_pixel_coords_all_landmarks(current_frame_keypoints_raw, frame_width, frame_height)

        # Draw connections
        for connection in mp_pose.POSE_CONNECTIONS:
            start_idx, end_idx = connection
            if start_idx in all_landmarks_pixel and end_idx in all_landmarks_pixel:
                cv2.line(annotated_frame, all_landmarks_pixel[start_idx], all_landmarks_pixel[end_idx], (0, 0, 255), 2) # Red connections

        # Draw landmarks as circles
        for idx, (x_px, y_px) in all_landmarks_pixel.items():
            cv2.circle(annotated_frame, (x_px, y_px), 3, (0, 255, 0), -1) # Green circles

    # 4b. Draw lines representing angles and add angle values as text
    if frame_idx < len(smoothed_keypoints_df):
        current_smoothed_keypoints = smoothed_keypoints_df.iloc[frame_idx]
        # Retrieve angles for the current frame
        current_angles = angles_df.iloc[frame_idx]

        # Need pixel coordinates for drawing, using the previously defined get_pixel_coords for relevant keypoints
        def get_pixel_coords_relevant(row, kp_name, frame_width, frame_height):
            # Ensure kp_name exists in the row (e.g., 'right_shoulder_x')
            if f'{kp_name}_x' in row and f'{kp_name}_y' in row:
                return (int(row[f'{kp_name}_x'] * frame_width),
                        int(row[f'{kp_name}_y'] * frame_height))
            return None


        # Only draw if angle is not NaN
        if pd.notna(current_angles.get('right_elbow_angle')):
            p_r_shoulder = get_pixel_coords_relevant(current_smoothed_keypoints, 'right_shoulder', frame_width, frame_height)
            p_r_elbow = get_pixel_coords_relevant(current_smoothed_keypoints, 'right_elbow', frame_width, frame_height)
            p_r_wrist = get_pixel_coords_relevant(current_smoothed_keypoints, 'right_wrist', frame_width, frame_height)

            if all(p is not None for p in [p_r_shoulder, p_r_elbow, p_r_wrist]):
                # Draw lines forming the angle
                cv2.line(annotated_frame, p_r_shoulder, p_r_elbow, (255, 255, 0), 2)
                cv2.line(annotated_frame, p_r_elbow, p_r_wrist, (255, 255, 0), 2)

                # Put angle text
                angle_text = f"R-Elbow: {current_angles['right_elbow_angle']:.1f}"
                cv2.putText(annotated_frame, angle_text, (p_r_elbow[0] + 10, p_r_elbow[1] - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 1, cv2.LINE_AA)

        if pd.notna(current_angles.get('left_elbow_angle')):
            p_l_shoulder = get_pixel_coords_relevant(current_smoothed_keypoints, 'left_shoulder', frame_width, frame_height)
            p_l_elbow = get_pixel_coords_relevant(current_smoothed_keypoints, 'left_elbow', frame_width, frame_height)
            p_l_wrist = get_pixel_coords_relevant(current_smoothed_keypoints, 'left_wrist', frame_width, frame_height)

            if all(p is not None for p in [p_l_shoulder, p_l_elbow, p_l_wrist]):
                cv2.line(annotated_frame, p_l_shoulder, p_l_elbow, (0, 255, 255), 2)
                cv2.line(annotated_frame, p_l_elbow, p_l_wrist, (0, 255, 255), 2)

                angle_text = f"L-Elbow: {current_angles['left_elbow_angle']:.1f}"
                cv2.putText(annotated_frame, angle_text, (p_l_elbow[0] + 10, p_l_elbow[1] - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 1, cv2.LINE_AA)

        if pd.notna(current_angles.get('right_shoulder_angle')):
            p_r_hip = get_pixel_coords_relevant(current_smoothed_keypoints, 'right_hip', frame_width, frame_height)
            p_r_shoulder = get_pixel_coords_relevant(current_smoothed_keypoints, 'right_shoulder', frame_width, frame_height)
            p_r_elbow = get_pixel_coords_relevant(current_smoothed_keypoints, 'right_elbow', frame_width, frame_height)

            if all(p is not None for p in [p_r_hip, p_r_shoulder, p_r_elbow]):
                cv2.line(annotated_frame, p_r_hip, p_r_shoulder, (255, 0, 255), 2)
                cv2.line(annotated_frame, p_r_shoulder, p_r_elbow, (255, 0, 255), 2)

                angle_text = f"R-Shoulder: {current_angles['right_shoulder_angle']:.1f}"
                cv2.putText(annotated_frame, angle_text, (p_r_shoulder[0] + 10, p_r_shoulder[1] - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 255), 1, cv2.LINE_AA)

        if pd.notna(current_angles.get('left_shoulder_angle')):
            p_l_hip = get_pixel_coords_relevant(current_smoothed_keypoints, 'left_hip', frame_width, frame_height)
            p_l_shoulder = get_pixel_coords_relevant(current_smoothed_keypoints, 'left_shoulder', frame_width, frame_height)
            p_l_elbow = get_pixel_coords_relevant(current_smoothed_keypoints, 'left_elbow', frame_width, frame_height)

            if all(p is not None for p in [p_l_hip, p_l_shoulder, p_l_elbow]):
                cv2.line(annotated_frame, p_l_hip, p_l_shoulder, (0, 165, 255), 2)
                cv2.line(annotated_frame, p_l_shoulder, p_l_elbow, (0, 165, 255), 2)

                angle_text = f"L-Shoulder: {current_angles['left_shoulder_angle']:.1f}"
                cv2.putText(annotated_frame, angle_text, (p_l_shoulder[0] + 10, p_l_shoulder[1] - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 165, 255), 1, cv2.LINE_AA)


    # 4c. Retrieve and display textual feedback messages
    feedback_messages = frame_feedback[frame_idx]['deviations'] if frame_idx < len(frame_feedback) else []
    y_offset = 30 # Initial vertical position for text
    for msg in feedback_messages:
        cv2.putText(annotated_frame, msg, (10, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2, cv2.LINE_AA)
        y_offset += 30

    # If no deviations, display 'Good Form!' or similar
    if not feedback_messages and frame_idx < len(smoothed_keypoints_df): # Only if keypoints were detected for the frame
         cv2.putText(annotated_frame, 'Good Form!', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2, cv2.LINE_AA)

    # 4d. Write the modified frame to the cv2.VideoWriter object
    out.write(annotated_frame)
    frame_idx += 1

# 5. Release the video capture and video writer objects
cap.release()
out.release()

print(f"Feedback video generated successfully: {output_video_path}")

# 6. Optionally, display the generated video within the Colab notebook
print("Displaying generated video:")
display(Video(output_video_path, embed=True, width=600))

In [ ]:
from collections import defaultdict

# 1. Aggregate deviations from the frame_feedback list
# 2. Count occurrences and calculate percentages

all_deviations_count = defaultdict(int)
frame_count_with_deviations = defaultdict(int)
total_frames = len(frame_feedback)

for entry in frame_feedback:
    for deviation_msg in entry['deviations']:
        all_deviations_count[deviation_msg] += 1
    # Count how many frames each unique deviation appeared in (only once per frame)
    for unique_deviation in set(entry['deviations']):
        frame_count_with_deviations[unique_deviation] += 1

# Prepare summary data
deviation_summary = []
for msg, total_occurrences in all_deviations_count.items():
    # Calculate percentage based on frames where it occurred, not total occurrences
    frames_with_this_deviation = frame_count_with_deviations[msg]
    percentage = (frames_with_this_deviation / total_frames) * 100
    deviation_summary.append({
        'deviation': msg,
        'total_occurrences': total_occurrences,
        'frames_affected': frames_with_this_deviation,
        'percentage_of_frames': percentage
    })

# Sort by percentage descending
deviation_summary.sort(key=lambda x: x['percentage_of_frames'], reverse=True)

print("Deviation Summary:")
for item in deviation_summary:
    print(f"- {item['deviation']}: Occurred in {item['frames_affected']}/{total_frames} frames ({item['percentage_of_frames']:.2f}%)")

In [ ]:
import datetime

# 3. Formulate actionable advice for improvement
# This can be expanded with more specific advice for various exercises and deviation types.
advice_map = {
    'Right arm not fully extended at start.': 'Ensure your right arm is fully extended at the bottom of the curl to maximize range of motion and engage the bicep fully.',
    'Left arm not fully extended at start.': 'Ensure your left arm is fully extended at the bottom of the curl to maximize range of motion and engage the bicep fully.',
    'Right shoulder not stable, arm swinging forward/backward.': 'Keep your right shoulder stable and tucked back. Avoid using momentum from your shoulder; focus on isolating the bicep.',
    'Left shoulder not stable, arm swinging forward/backward.': 'Keep your left shoulder stable and tucked back. Avoid using momentum from your shoulder; focus on isolating the bicep.',
    'Right elbow not fully contracted.': 'Bring your right hand up higher to achieve a full bicep contraction. Focus on squeezing the bicep at the top of the movement.',
    'Left elbow not fully contracted.': 'Bring your left hand up higher to achieve a full bicep contraction. Focus on squeezing the bicep at the top of the movement.',
    'Right arm not fully extended at bottom.': 'Ensure your right arm extends completely at the bottom of the movement. This ensures a full range of motion and proper muscle engagement.',
    'Left arm not fully extended at bottom.': 'Ensure your left arm extends completely at the bottom of the movement. This ensures a full range of motion and proper muscle engagement.',
    'Right shoulder moving too much (swinging).': 'Minimize right shoulder movement. Keep your elbows close to your body and focus on curling with your biceps, not swinging with your shoulders.',
    'Left shoulder moving too much (swinging).': 'Minimize left shoulder movement. Keep your elbows close to your body and focus on curling with your biceps, not swinging with your shoulders.'
}

# 4. Structure the information into a readable textual report
report_content = []

report_content.append("Fitness Form Analysis Report")
report_content.append("Date: " + datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
report_content.append("---------------------------------------------------")
report_content.append("\nOverall Performance Summary:")
if not deviation_summary:
    report_content.append("  Great job! No significant deviations were detected in your workout.")
else:
    report_content.append("  Several deviations were observed during your workout. Review the specific issues and advice below to improve your form.")

report_content.append("\nSpecific Issues Detected:")
if not deviation_summary:
    report_content.append("  None.")
else:
    for item in deviation_summary:
        report_content.append(f"  - {item['deviation']} (Observed in {item['percentage_of_frames']:.2f}% of frames)")

report_content.append("\nPersonalized Corrective Advice:")
if not deviation_summary:
    report_content.append("  Keep up the good work!")
else:
    for item in deviation_summary:
        deviation_message = item['deviation'].split(': ')[-1] # Extract just the deviation message part
        advice = advice_map.get(deviation_message, "No specific advice available for this deviation. Consult a trainer.")
        report_content.append(f"  - For '{deviation_message}': {advice}")

report_content.append("\n---------------------------------------------------")
report_content.append("Recommendations:")
report_content.append("  - Review the visual feedback video for frame-by-frame guidance.")
report_content.append("  - Focus on slow, controlled movements to maintain proper form throughout the exercise.")
report_content.append("  - Consider using lighter weights if form is consistently breaking down.")
report_content.append("  - Practice in front of a mirror to self-correct in real-time.")

final_report_text = "\n".join(report_content)

# 5. Save the generated report to a text file
report_filename = "workout_report.txt"
with open(report_filename, "w") as f:
    f.write(final_report_text)

print(f"Detailed textual report saved to {report_filename}")
print("\n--- Report Content Preview ---")
print(final_report_text)


In [ ]:
import os

# 1. Define the output_video_path variable
output_video_path = '/content/workout_feedback_video.mp4'

# 2. Check if the file exists
if os.path.exists(output_video_path):
    # 3. Get file size if it exists
    file_size_bytes = os.path.getsize(output_video_path)
    file_size_mb = file_size_bytes / (1024 * 1024) # Convert bytes to MB
    print(f"The feedback video '{output_video_path}' exists.")
    print(f"File size: {file_size_mb:.2f} MB")
else:
    print(f"Error: The feedback video '{output_video_path}' does not exist.")

In [ ]:
from google.colab import files

# Assuming output_video_path is already defined from the previous cell
# If not, uncomment and define it:
# output_video_path = '/content/workout_feedback_video.mp4'

if os.path.exists(output_video_path):
    print(f"\nGenerating download link for '{output_video_path}'...")
    files.download(output_video_path)
else:
    print(f"Error: Cannot generate download link. The file '{output_video_path}' does not exist.")